In [2]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import sys
import numpy as np
import glob
import torch

sys.path.insert(1,"/home/showalte/research/prob_seq_queries/")
from seq_queries.utils import read_pkl, write_pkl
from seq_queries.data import *

In [3]:
df, vocab = prepare_mobile_app_data_by_user('../../data/apps/lsapp.tsv',15)

In [13]:
cnts = np.unique(data,return_counts = True)

In [6]:
data = np.array(df)

In [18]:
cnts = [cnts[0][0], cnts[1][0]]

In [22]:
new_cnts.sort(key = lambda item: item[1], reverse=True)

In [23]:
new_cnts

[('Facebook', 7579565),
 ('Google Chrome', 4808567),
 ('Facebook Messenger', 3559963),
 ('Google', 3144866),
 ('Twitter', 2793944),
 ('Messages', 2356451),
 ('Instagram', 1888613),
 ('Reddit', 1844606),
 ('Swagbucks Watch (TV)', 1605846),
 ('Gmail', 1605023),
 ('WhatsApp Messenger', 1438127),
 ('Settings', 1350706),
 ('Verizon Messages', 1295973),
 ('Google Play Store', 1234015),
 ('Contacts', 988511),
 ('Phone', 969351),
 ('Telegram', 889252),
 ('Slidejoy', 748553),
 ('YouTube', 740648),
 ('Messaging', 707231),
 ('Samsung Internet Browser', 627971),
 ('Lucktastic', 467281),
 ('Maps', 454192),
 ('Samsung Email', 441105),
 ('Android In Call UI', 437935),
 ('Hangouts', 425379),
 ('WeChat', 412630),
 ('AOL', 380359),
 ('Telegram X', 356034),
 ('Movie Play Box', 336330),
 ('Swagbucks', 327360),
 ('Amazon Shopping', 325959),
 ('Samsung Gallery', 318086),
 ('OfferUp', 303240),
 ('Camera', 302582),
 ('SurveyCow', 293500),
 ('Quora', 292431),
 ('Microsoft Bing Search', 267450),
 ('Calendar', 2

In [20]:
new_cnts = list(zip(cnts[0].tolist(),cnts[1].tolist()))
new_cnts

[('AOL', 380359),
 ('Amazon Shopping', 325959),
 ('Android In Call UI', 437935),
 ('Army Men Strike', 177186),
 ('Badoo', 104445),
 ('Baseball Boy!', 35490),
 ('Brave Browser', 330),
 ('Calculator', 150720),
 ('Calendar', 253621),
 ('Calorie Counter', 157018),
 ('Camera', 302582),
 ('Clean Master', 49028),
 ('Clock', 170690),
 ('Contacts', 988511),
 ('DigiHUD Pro Speedometer', 120570),
 ('Discord', 107887),
 ('EntertaiNow', 133050),
 ('Facebook', 7579565),
 ('Facebook Messenger', 3559963),
 ('Faceu', 168584),
 ('Flickr', 360),
 ('Flipboard Briefing', 24778),
 ('Gmail', 1605023),
 ('Google', 3144866),
 ('Google Chrome', 4808567),
 ('Google Drive', 182474),
 ('Google Photos', 157504),
 ('Google Play Music', 185085),
 ('Google Play Store', 1234015),
 ('Hangouts', 425379),
 ('Hulu', 77688),
 ('Ibotta', 108476),
 ('Instagram', 1888613),
 ('Kik', 18270),
 ('Lucktastic', 467281),
 ('MAX Cleaner', 220065),
 ('MUIQ Survey App', 104752),
 ('Maps', 454192),
 ('Messages', 2356451),
 ('Messaging', 

In [9]:
# cnts = []
# for i in range(88):
#     cnts.append((data == i).astype(int).sum())
    

<ipython-input-9-b4e3c27b1cfd>:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  cnts.append((data == i).astype(int).sum())


AttributeError: 'bool' object has no attribute 'astype'

In [ ]:
plt.bar(range(cnts),cnts)

In [2]:
os.chdir("../")
df = pd.read_csv('data/lsapp.tsv', sep='\t')
df_open = df.loc[df['event_type'] == 'Opened']

In [3]:
df_open.loc[:,'timestamp'] = pd.to_datetime(df_open['timestamp'])
df.loc[:,'timestamp'] = pd.to_datetime(df['timestamp'])

<ipython-input-3-6cb7748109c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_open.loc[:,'timestamp'] = pd.to_datetime(df_open['timestamp'])


In [7]:
df.head()
df = df.sort_values(by=['user_id',
                        'session_id',
                        'timestamp',
                        'event_type'])
# df.head()
print(df.columns)
ndf = df.loc[:,['user_id','session_id','app_name']].values
# df
                    
    
# df.user_id.drop_duplicates()
# df.shape

Index(['user_id', 'session_id', 'timestamp', 'app_name', 'event_type'], dtype='object')


In [26]:
def read_mobile_app_data(data_path):
    df = pd.read_csv(data_path, sep='\t')
    df.loc[:,'timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(by=['user_id',
                        'session_id',
                        'timestamp'])
    return df.loc[:,['user_id','session_id','app_name']].values

def stratify_app_data_by_user(df):
    seqs = []; curr_seq = []
    curr_user, curr_session, token = df[0]
    curr_seq.append(token)
    for i in tqdm(range(df.shape[0])):
        user, session,token = df[i]
        # New user and new session
        if user != curr_user:
            curr_user = user
            seqs.append(curr_seq)
            curr_seq = []
        # Just a new session, same user
        elif session != curr_session:
            curr_session = session
        curr_seq.append(token)
    
    return seqs
    
def get_app_sequences(df_list,
                      seq_len,
):
    flat_seqs = []
    for user_data in df_list:
        if len(user_data) < seq_len:
            continue
        for i in range(seq_len,len(user_data),1):
            flat_seqs.append(user_data[i-seq_len,i])
    return flat_seqs

def prepare_mobile_app_data_by_user(
    data_path,
    seq_len,
):
    
    df = read_mobile_app_data(data_path)
    df_list = stratify_app_data_by_user(df)
    df_sequences = get_app_sequences(df_list)
    return df_sequences
    
sequences = prepare_mobile_app_data_by_user(ndf)
print(sorted([len(s) for s in sequences]))

100%|██████████| 3658589/3658589 [00:03<00:00, 1003889.07it/s]

[2, 6, 12, 22, 22, 26, 34, 40, 41, 42, 49, 60, 84, 87, 110, 120, 121, 145, 158, 167, 173, 176, 186, 190, 194, 196, 196, 214, 225, 237, 242, 250, 253, 284, 290, 290, 295, 298, 307, 330, 358, 363, 363, 366, 376, 411, 420, 427, 461, 462, 465, 466, 474, 508, 515, 516, 519, 525, 537, 547, 547, 554, 559, 561, 612, 616, 657, 685, 709, 710, 742, 754, 758, 771, 784, 797, 807, 825, 849, 906, 928, 934, 945, 974, 992, 1010, 1024, 1063, 1095, 1110, 1145, 1148, 1173, 1190, 1196, 1197, 1215, 1240, 1261, 1278, 1299, 1369, 1374, 1387, 1392, 1436, 1496, 1557, 1560, 1570, 1576, 1604, 1660, 1727, 1768, 1841, 1852, 1853, 1901, 1963, 2135, 2204, 2244, 2250, 2270, 2319, 2331, 2368, 2420, 2470, 2505, 2518, 2518, 2571, 2579, 2667, 2811, 2819, 2919, 2922, 2947, 2961, 3025, 3116, 3192, 3249, 3318, 3332, 3362, 3381, 3414, 3424, 3451, 3486, 3525, 3620, 3627, 3817, 3830, 3909, 4098, 4118, 4152, 4166, 4247, 4251, 4264, 4401, 4475, 4496, 4577, 4585, 4621, 4790, 5053, 5117, 5147, 5195, 5269, 5292, 5431, 5487, 5580, 56

In [11]:
# print(sequences[0])

In [13]:
df['time_dff']  = df[['timestamp']].diff()
df['interaction_id'] = (((df.timestamp-df.timestamp.shift(1) > pd.Timedelta(1, 'm')) 
                              & (df.event_type == 'Opened')) 
                              | ~(df.app_name == df.app_name.shift(1))
                              | ~(df.user_id == df.user_id.shift(1))).cumsum()
df['session_id'] = (((df.timestamp-df.timestamp.shift(1) > pd.Timedelta(5, 'm')) 
                              & (df.event_type == 'Opened')) 
                              | ~(df.user_id == df.user_id.shift(1))).cumsum()
df_start = df.drop_duplicates(subset=['interaction_id'], keep='first')
df_end = df.drop_duplicates(subset=['interaction_id'], keep='last')
df_start.set_index('interaction_id', inplace=True)
df_end.set_index('interaction_id', inplace=True)
df_start.loc[:, 'open_time'] = df_start['timestamp']
df_start.loc[:, 'close_time']  = df_end['timestamp']

<ipython-input-13-ae8a15f1f18d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_start.loc[:, 'open_time'] = df_start['timestamp']
<ipython-input-13-ae8a15f1f18d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_start.loc[:, 'close_time']  = df_end['timestamp']


In [17]:
df_start.groupby('event_type').count()

,user_id,session_id,timestamp,app_name,time_dff,open_time,close_time
event_type,,,,,,,
Broken,259,259,259,259,259,259,259
Closed,146261,146261,146261,146261,146261,146261,146261
Opened,359143,359143,359143,359143,359142,359143,359143
User Interaction,93972,93972,93972,93972,93972,93972,93972
